In [2]:
cd ../..

/media/atem/Data/HSE_videos/4_DLA/hw_2_SeppechSep/git_speech_separation


In [6]:
!python3 inference.py --config-name=inference model=dttnet model_loader.from_pretrained="areshx/source-sep@dttnet_low_freq"

[2025-11-21 11:11:08,019][__main__][INFO] - Loading pretrained model from Hugging Face: areshx/source-sep@dttnet_low_freq
[2025-11-21 11:11:08,019][__main__][INFO] - Downloading model from Hugging Face Hub: areshx/source-sep@dttnet_low_freq (model.pth)
[2025-11-21 11:11:08,672][__main__][INFO] - Model weights successfully loaded from Hugging Face: areshx/source-sep@dttnet_low_freq
[2025-11-21 11:11:08,673][__main__][INFO] - DTTNetModel(
  (encoder): Encoder(
    (init_conv): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1))
    (encoder_layers): ModuleList(
      (0): EncoderBlock(
        (tfc_tdf): TFC_TDF_Block(
          (conv1): Sequential(
            (0): Sequential(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
              (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
              (2): GELU(approximate='none')
            )
            (1): Sequential(
              (0): Conv2d(32, 32, kernel_siz